In [2]:
import torchvision.transforms as transforms

In [4]:
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (1.0, ))
])

In [5]:
from torchvision.datasets import MNIST
import requests

In [7]:
download_root = 'C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data'

In [8]:
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw\train-images-idx3-ubyte.gz to C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw\train-labels-idx1-ubyte.gz to C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw\t10k-images-idx3-ubyte.gz to C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data\MNIST\raw



---

In [11]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/165.9 kB ? eta -:--:--
     ------- -------------------------------- 30.7/165.9 kB ? eta -:--:--
     -------------------------------------- 165.9/165.9 kB 5.0 MB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.0 MB 20.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.0/8.0 MB 25.2 MB/s eta 0:00:01
   ---------------- ----------------------- 3.4/8.0 MB 26.8 MB/s eta 0:00:01
   ------------------ --------------------- 3.6/8.0 MB 19.2 MB/s eta 0:00:01
   --------------------------- ------------ 5.4/8.0 MB 23.0 MB/s eta 0:00:01
   ------------------------------------ --- 7.4/8.0 MB 26.1 MB/s eta 0:00:01
   ---------------------------------

In [1]:
import numpy as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
download_root = 'C:/Users/SKT01/Desktop/강의자료/Pytorch with AI/AI융합프로그램/data'

In [4]:
train_dataset = torchvision.datasets.FashionMNIST(download_root, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.FashionMNIST(download_root, download=True, train=False, transform=transforms.Compose([transforms.ToTensor()]))

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100)

In [6]:
labels_map = {
    0: 'T-Shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot'
}

In [7]:
class FashionDNN(nn.Module):
    def __init__(self):
        super(FashionDNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [8]:
learning_rate = 0.001
model = FashionDNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

FashionDNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [10]:
'''
num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
    
    train = Variable(images.view(images.size(0), 1, 28, 28))
    labels = Variable(labels)

    outputs = model(train)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    count += 1

    if not (count % 50):
        total = 0
        correct = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels_list.append(labels)
            test = Variable(images.view(images.size(0), 1, 28, 28))
            outputs = model(test)
            predictions = torch.max(outputs, 1)[1].to(device)
            predictions_list.append(predictions)
            correct += (predictions == labels).sum().item()
            total += len(labels)
        
        accuracy = correct * 100 / total
        loss_list.append(loss.data)
        iteration_list.append(count)
        accuracy_list.append(accuracy)

    if not (count % 500):
        print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))
    '''

c:\Users\SKT01\anaconda3\envs\torch_book\lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [11]:
'''
num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        train = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 train_loader와 맞추기 위해 수정
        labels = labels

        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1

        if count % 50 == 0:
            total = 0
            correct = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    test = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 test_loader와 맞추기 위해 수정
                    outputs = model(test)
                    predictions = torch.max(outputs, 1)[1]
                    correct += (predictions == labels).sum().item()
                    total += labels.size(0)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

print("Training complete!")
'''

Iteration: 50, Loss: 0.42750126123428345, Accuracy: 79.89%
Iteration: 100, Loss: 0.475765585899353, Accuracy: 84.51%
Iteration: 150, Loss: 0.42978447675704956, Accuracy: 84.5%
Iteration: 200, Loss: 0.43274906277656555, Accuracy: 85.19%
Iteration: 250, Loss: 0.4499438405036926, Accuracy: 83.4%
Iteration: 300, Loss: 0.30584055185317993, Accuracy: 87.62%
Iteration: 350, Loss: 0.3765990436077118, Accuracy: 86.33%
Iteration: 400, Loss: 0.3990320861339569, Accuracy: 82.69%
Iteration: 450, Loss: 0.6188592910766602, Accuracy: 85.53%
Iteration: 500, Loss: 0.4051140248775482, Accuracy: 87.31%
Iteration: 550, Loss: 0.3517197370529175, Accuracy: 88.22%
Iteration: 600, Loss: 0.2855070233345032, Accuracy: 86.78%
Iteration: 650, Loss: 0.29856136441230774, Accuracy: 88.45%
Iteration: 700, Loss: 0.2577352821826935, Accuracy: 88.8%
Iteration: 750, Loss: 0.370523601770401, Accuracy: 86.62%
Iteration: 800, Loss: 0.2858850955963135, Accuracy: 87.93%
Iteration: 850, Loss: 0.32631218433380127, Accuracy: 86.9

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

best_accuracy = 0.0
best_model_wts = None

# 훈련 데이터 크기와 배치 크기
train_data_size = len(train_loader.dataset)
batch_size = train_loader.batch_size

# iteration 수 계산
total_iterations = num_epochs * (train_data_size // batch_size)

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        train = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 train_loader와 맞추기 위해 수정
        labels = labels

        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1

        if count % 50 == 0:
            total = 0
            correct = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    test = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 test_loader와 맞추기 위해 수정
                    outputs = model(test)
                    predictions = torch.max(outputs, 1)[1]
                    correct += (predictions == labels).sum().item()
                    total += labels.size(0)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            print(f"Iteration: {count}/{total_iterations}, Loss: {loss.item()}, Accuracy: {accuracy}%")

            # 가장 높은 정확도를 기록한 모델 저장
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model_wts = model.state_dict()

# 최종 모델 저장
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_model.pth')
    print(f"Best model saved with accuracy: {best_accuracy}%")

print("Training complete!")


c:\Users\SKT01\anaconda3\envs\torch_book\lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Iteration: 50/3000, Loss: 0.16564151644706726, Accuracy: 91.09%
Iteration: 100/3000, Loss: 0.12494651973247528, Accuracy: 90.09%
Iteration: 150/3000, Loss: 0.23480354249477386, Accuracy: 90.58%
Iteration: 200/3000, Loss: 0.22143825888633728, Accuracy: 89.77%
Iteration: 250/3000, Loss: 0.20013116300106049, Accuracy: 89.21%
Iteration: 300/3000, Loss: 0.1350240856409073, Accuracy: 90.18%
Iteration: 350/3000, Loss: 0.19082732498645782, Accuracy: 90.01%
Iteration: 400/3000, Loss: 0.2076500654220581, Accuracy: 90.43%
Iteration: 450/3000, Loss: 0.44753074645996094, Accuracy: 90.63%
Iteration: 500/3000, Loss: 0.22584658861160278, Accuracy: 90.78%
Iteration: 550/3000, Loss: 0.21136701107025146, Accuracy: 90.02%
Iteration: 600/3000, Loss: 0.1873020976781845, Accuracy: 90.51%
Iteration: 650/3000, Loss: 0.1611260026693344, Accuracy: 90.81%
Iteration: 700/3000, Loss: 0.10839495062828064, Accuracy: 90.73%
Iteration: 750/3000, Loss: 0.26778653264045715, Accuracy: 90.69%
Iteration: 800/3000, Loss: 0.1

In [25]:
class newFashionDNN(nn.Module):
    def __init__(self):
        super(newFashionDNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=128*3*3, out_features=256)
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [26]:
learning_rate = 0.001
model = newFashionDNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

newFashionDNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=256, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
  (fc

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

best_accuracy = 0.0
best_model_wts = None

# 훈련 데이터 크기와 배치 크기
train_data_size = len(train_loader.dataset)
batch_size = train_loader.batch_size

# iteration 수 계산
total_iterations = num_epochs * (train_data_size // batch_size)

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        train = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 train_loader와 맞추기 위해 수정
        labels = labels

        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1

        if count % 50 == 0:
            total = 0
            correct = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    test = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 test_loader와 맞추기 위해 수정
                    outputs = model(test)
                    predictions = torch.max(outputs, 1)[1]
                    correct += (predictions == labels).sum().item()
                    total += labels.size(0)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            print(f"Iteration: {count}/{total_iterations}, Loss: {loss.item()}, Accuracy: {accuracy}%")

            # 가장 높은 정확도를 기록한 모델 저장
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model_wts = model.state_dict()

# 최종 모델 저장
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_model.pth')
    print(f"Best model saved with accuracy: {best_accuracy}%")

print("Training complete!")


Iteration: 50/3000, Loss: 0.5421976447105408, Accuracy: 79.53%
Iteration: 100/3000, Loss: 0.5422972440719604, Accuracy: 80.81%
Iteration: 150/3000, Loss: 0.47840631008148193, Accuracy: 82.54%
Iteration: 200/3000, Loss: 0.4011068642139435, Accuracy: 84.73%
Iteration: 250/3000, Loss: 0.30903682112693787, Accuracy: 85.39%
Iteration: 300/3000, Loss: 0.44430840015411377, Accuracy: 83.43%
Iteration: 350/3000, Loss: 0.35734233260154724, Accuracy: 86.07%
Iteration: 400/3000, Loss: 0.398668110370636, Accuracy: 87.19%
Iteration: 450/3000, Loss: 0.49934250116348267, Accuracy: 86.15%
Iteration: 500/3000, Loss: 0.4159265160560608, Accuracy: 87.25%
Iteration: 550/3000, Loss: 0.2948434352874756, Accuracy: 86.96%
Iteration: 600/3000, Loss: 0.28693127632141113, Accuracy: 86.27%
Iteration: 650/3000, Loss: 0.29102623462677, Accuracy: 88.31%
Iteration: 700/3000, Loss: 0.2072339802980423, Accuracy: 88.74%
Iteration: 750/3000, Loss: 0.3002639412879944, Accuracy: 87.82%
Iteration: 800/3000, Loss: 0.213554874

In [22]:
class BestFashionCNN(nn.Module):
    
    def __init__(self):
        super(BestFashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [23]:
learning_rate = 0.001
model = BestFashionCNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

BestFashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

best_accuracy = 0.0
best_model_wts = None

# 훈련 데이터 크기와 배치 크기
train_data_size = len(train_loader.dataset)
batch_size = train_loader.batch_size

# iteration 수 계산
total_iterations = num_epochs * (train_data_size // batch_size)

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        train = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 train_loader와 맞추기 위해 수정
        labels = labels

        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1

        if count % 50 == 0:
            total = 0
            correct = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    test = images.view(images.size(0), 1, 28, 28)  # 배치 크기를 test_loader와 맞추기 위해 수정
                    outputs = model(test)
                    predictions = torch.max(outputs, 1)[1]
                    correct += (predictions == labels).sum().item()
                    total += labels.size(0)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            print(f"Iteration: {count}/{total_iterations}, Loss: {loss.item()}, Accuracy: {accuracy}%")

            # 가장 높은 정확도를 기록한 모델 저장
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model_wts = model.state_dict()

# 최종 모델 저장
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_model.pth')
    print(f"Best model saved with accuracy: {best_accuracy}%")

print("Training complete!")

Iteration: 50/3000, Loss: 0.5406960248947144, Accuracy: 79.64%
Iteration: 100/3000, Loss: 0.46793416142463684, Accuracy: 80.31%
Iteration: 150/3000, Loss: 0.4378383159637451, Accuracy: 84.9%
Iteration: 200/3000, Loss: 0.3999532461166382, Accuracy: 85.45%
Iteration: 250/3000, Loss: 0.4046562910079956, Accuracy: 86.19%
Iteration: 300/3000, Loss: 0.4444766640663147, Accuracy: 85.83%
Iteration: 350/3000, Loss: 0.34449490904808044, Accuracy: 84.55%
Iteration: 400/3000, Loss: 0.4054742753505707, Accuracy: 85.48%
Iteration: 450/3000, Loss: 0.6262008547782898, Accuracy: 84.82%
Iteration: 500/3000, Loss: 0.4640432596206665, Accuracy: 87.94%
Iteration: 550/3000, Loss: 0.4052322506904602, Accuracy: 88.08%
Iteration: 600/3000, Loss: 0.24692806601524353, Accuracy: 86.58%
Iteration: 650/3000, Loss: 0.2659952938556671, Accuracy: 88.9%
Iteration: 700/3000, Loss: 0.2456912398338318, Accuracy: 88.52%
Iteration: 750/3000, Loss: 0.3559759855270386, Accuracy: 87.03%
Iteration: 800/3000, Loss: 0.34948366880

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 최적화된 newFashionDNN 모델 정의
class newFashionDNN(nn.Module):
    def __init__(self):
        super(newFashionDNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=128*3*3, out_features=256)
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# 데이터셋 및 데이터 로더 설정
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 모델, 손실 함수, 옵티마이저 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = newFashionDNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 스케줄러 초기화
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 훈련 설정
num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

best_accuracy = 0.0
best_model_wts = None

# 훈련 데이터 크기와 배치 크기
train_data_size = len(train_loader.dataset)

# iteration 수 계산
total_iterations = num_epochs * (train_data_size // batch_size)

for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1

        if count % 50 == 0:
            model.eval()  # 모델을 평가 모드로 설정
            total = 0
            correct = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    predictions = torch.max(outputs, 1)[1]
                    correct += (predictions == labels).sum().item()
                    total += labels.size(0)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            print(f"Iteration: {count}/{total_iterations}, Loss: {loss.item()}, Accuracy: {accuracy}%")

            # 가장 높은 정확도를 기록한 모델 저장
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model_wts = model.state_dict()

    # 스케줄러 업데이트
    scheduler.step()
    model.train()  # 다음 epoch를 위해 모델을 다시 훈련 모드로 설정

# 최종 모델 저장
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_model.pth')
    print(f"Best model saved with accuracy: {best_accuracy}%")

print("Training complete!")


Iteration: 50/4685, Loss: 0.45976966619491577, Accuracy: 73.46%
Iteration: 100/4685, Loss: 0.6057692766189575, Accuracy: 80.88%
Iteration: 150/4685, Loss: 0.43997037410736084, Accuracy: 82.66%
Iteration: 200/4685, Loss: 0.4657905697822571, Accuracy: 83.9%
Iteration: 250/4685, Loss: 0.38294345140457153, Accuracy: 85.9%
Iteration: 300/4685, Loss: 0.19038014113903046, Accuracy: 85.55%
Iteration: 350/4685, Loss: 0.3014202117919922, Accuracy: 86.27%
Iteration: 400/4685, Loss: 0.49786195158958435, Accuracy: 85.94%
Iteration: 450/4685, Loss: 0.2876548767089844, Accuracy: 87.28%
Iteration: 500/4685, Loss: 0.38239458203315735, Accuracy: 87.52%
Iteration: 550/4685, Loss: 0.4319179952144623, Accuracy: 88.09%
Iteration: 600/4685, Loss: 0.29250869154930115, Accuracy: 88.0%
Iteration: 650/4685, Loss: 0.16631552577018738, Accuracy: 88.19%
Iteration: 700/4685, Loss: 0.3579227030277252, Accuracy: 87.63%
Iteration: 750/4685, Loss: 0.3134734332561493, Accuracy: 87.12%
Iteration: 800/4685, Loss: 0.1630764